In [25]:
from langgraph.graph import StateGraph,START,END
from pydantic import BaseModel

class GraphState1(BaseModel):
    a: int = 0
    b: int = 0

def graph1_node1(state: GraphState1):
    a=state.a
    a+=1
    return {"a":a}
def graph1_node2(state: GraphState1):
    b=state.b
    b+=1
    return {"b":b}


graph1=StateGraph(GraphState1)
graph1.add_node(graph1_node1.__name__,graph1_node1)
graph1.add_node(graph1_node2.__name__,graph1_node2)
graph1.add_edge(START,graph1_node1.__name__)
graph1.add_edge(graph1_node1.__name__,graph1_node2.__name__)
graph1.add_edge(graph1_node2.__name__,END)
graph1_app= graph1.compile()

async for chunk in graph1_app.astream({"a":0,"b":0}):
    print(chunk)

{'graph1_node1': {'a': 1}}
{'graph1_node2': {'b': 1}}


In [27]:
class GraphState2(BaseModel):
    c: int = 0
    d: int = 0

async def graph2_node1(state: GraphState2):
    async for chunk in graph1_app.astream({"a":0,"b":0},stream_mode="updates"):
        print(chunk)
    c=state.c
    c+=1
    return {"c":c}
def graph2_node2(state: GraphState2):
    d=state.d
    d+=1
    return {"d":d}

graph2=StateGraph(GraphState2)
graph2.add_node(graph2_node1.__name__,graph2_node1)
graph2.add_node(graph2_node2.__name__,graph2_node2)
graph2.add_edge(START,graph2_node1.__name__)
graph2.add_edge(graph2_node1.__name__,graph2_node2.__name__)
graph2.add_edge(graph2_node2.__name__,END)
graph2_app= graph2.compile()

async for chunk in graph2_app.astream_events({"c":0,"d":0},version="v2"):
    pass

{'a': 0, 'b': 0}
{'a': 1, 'b': 0}
{'a': 1, 'b': 1}


In [23]:
from pydantic_core  import from_json

json_string="""{"a":1,"b":"123\n123"}"""

json_result=from_json(json_string)

print(json_result)

ValueError: control character (\u0000-\u001F) found while parsing a string at line 2 column 0